In [29]:
import os
import pandas as pd
import numpy as np
import datetime

**A**

In [356]:
while True:
    # Request user input
    A = input("What is the phenology of the potato field?\n \
                 A) Pre-emergence\n \
                 B) Emergence to before plant closure between rows\n \
                 C) Plant closure between rows (foliage touching between adjacent rows)\n \
                 D) Post row closure to harvest\n")
    # If Risk Factor (RF) A is option "A" or option "B"
    if (A == "A") or (A == "B"):
        # Set RF to 0
        A = 0
    # Else, RF is "C" or "D" and
    elif (A == "C") or (A == "D"):
        # Set it to 1
        A = 1
    # Break
    break

What is the phenology of the potato field?
                  A) Pre-emergence
                  B) Emergence to before plant closure between rows
                  C) Plant closure between rows (foliage touching between adjacent rows)
                  D) Post row closure to harvest
 B


**B**

In [350]:
while True:
    # Request user input
    B = input("Is late blight present within a 5 mile radius?\n \
                 A) No\n \
                 B) Yes\n")
    # If Risk Factor (RF) B is option "A"
    if (B == "A") :
        # Set RF to 0
        B = 0
    # Else, RF is "B" and
    elif (B == "B"):
        # Set it to 30
        B = 30
    # Break
    break

Is late blight present within a 5 mile radius?
                  A) No
                  B) Yes
 B


**C**

In [306]:
os.chdir('C:/Users/sudha.gcupadhaya/Desktop/LateBlight/Data')
df_othello = pd.read_csv('Othello_A_2020.csv')
df_prosser = pd.read_csv('Prosser_A_2020.csv')
df_tricities = pd.read_csv('Tricities_A_2020.csv')

In [307]:
df_othello['Station'] = 'Othello'
df_prosser['Station'] = 'Prosser'
df_tricities['Station'] = 'Tricities'

In [308]:
df_AM = pd.concat([df_othello, df_prosser, df_tricities])
df_AM.head()

,Date,Date.1,Min°C,Avg°C,Max°C,Avg1.5m DP°C,Avg1.5m RH%,Avg1.5m LWu.,AvgDir,Avg Speedm/s,2m MaxGustm/s,5 cm°C,Min°C.1,Avg°C.1,TotPrecmm,TotalSolarRadMJ/m²,ETomm,ETrmm,Station,Avg2m Atm.PresshPa
0,2020/04/01,1,-2.9,3.0,9.5,-2.1,72.0,0.02,W,2.6,7.0,NaN,6.1,7.3,0.0,15.50,1.97,2.64,Othello,NaN
1,2020/04/02,2,-4.3,3.2,10.2,-6.4,53.2,0.01,W,2.4,7.3,NaN,5.3,6.9,0.0,19.53,2.54,3.50,Othello,NaN
2,2020/04/03,3,-2.9,3.8,11.1,-5.3,56.7,0.00,SW,4.5,12.8,NaN,5.7,7.2,0.0,19.06,3.07,4.48,Othello,NaN
3,2020/04/04,4,-3.6,5.0,11.1,-5.4,51.0,0.00,NE,1.7,6.5,NaN,5.6,7.4,0.0,19.59,2.44,3.21,Othello,NaN
4,2020/04/05,5,3.0,7.4,13.5,1.3,68.3,0.00,N,2.4,6.8,NaN,7.6,8.7,0.0,14.91,2.47,3.33,Othello,NaN


In [310]:
df_AM = df_AM[['Date', 'TotPrecmm','Station']]#subset only three columns
print(df_AM.shape)
df_AM.head()

(90, 3)


,Date,TotPrecmm,Station
0,2020/04/01,0.0,Othello
1,2020/04/02,0.0,Othello
2,2020/04/03,0.0,Othello
3,2020/04/04,0.0,Othello
4,2020/04/05,0.0,Othello


In [315]:
grouped = df_AM.groupby(['Station','TotPrecmm']).size().reset_index(name = 'PrecCount')
grouped

,Station,TotPrecmm,PrecCount
0,Othello,0.00,29
1,Othello,1.52,1
2,Prosser,0.00,29
3,Prosser,1.78,1
4,Tricities,0.00,29
5,Tricities,1.02,1


In [317]:
grouped = grouped.query('TotPrecmm==0')
grouped['RainyDays'] = 30- grouped['PrecCount']
grouped

,Station,TotPrecmm,PrecCount,RainyDays
0,Othello,0.0,29,1
2,Prosser,0.0,29,1
4,Tricities,0.0,29,1


In [320]:
RainDays_April_Othello = np.array(grouped['RainyDays'][(grouped['Station']=='Othello')])
RainDays_April_Prosser = np.array(grouped['RainyDays'][(grouped['Station']=='Prosser')])
RainDays_April_Tricities = np.array(grouped['RainyDays'][(grouped['Station']=='Tricities')])

* Just random random for now

In [ ]:
RainDays_May_Othello = 6
RainDays_May_Prosser = 9
RainDays_May_Tricities = 10

#### Model 1: logistic regression

- **Prosser**

> $lf = 9.252 - 4.004(Y_p) - 0.660(R_{am})$

> where:

> $Y_p$: late blight outbreak during the preceding year:
$$  
Y_p
\begin{cases}
  \text{late blight absent in previous year/ no} = 0 \\    
  \text{late blight present in previous year/ yes} = 1 
\end{cases}
$$
> $R_{am}$: number of days with rain >= 0.25 mm during April and May

In [343]:
lf_Prosser = 9.252 - 4.004*1 - 0.660*(RainDays_April_Prosser+RainDays_May_Prosser)
print(lf_Prosser)
Prob_Prosser= 1 / (1 + np.exp(lf_Prosser))
print('Probability of late blight occurence in Prosser:')
print(Prob_Prosser)

[-1.352]
Probability of late blight occurence in Prosser:
[0.79445641]


- **Tri-Cities**

> $lf = -2.987 - 3.062(Y_p) - 0.163(R_{am})$

In [346]:
lf_Tricities = -2.987 - 3.062*1 - 0.163*(RainDays_April_Tricities + RainDays_May_Tricities)
print(lf_Tricities)
Prob_Tricities = 1 / (1 + np.exp(lf_Tricities))
print('Probability of late blight occurence in Tricites:')
print(Prob_Tricities)

[-7.842]
Probability of late blight occurence in Tricites:
[0.99960727]


- **Othello**

> $lf = 4.723 - 3.561(Y_p) - 0.293(R_{am})$

In [348]:
lf_Othello = 4.723 - 3.561*1 - 0.293*(RainDays_April_Othello + RainDays_May_Othello)
print(lf_Othello)
Prob_Othello = 1 / (1 + np.exp(lf_Othello))
print('Probability of late blight occurence in Othello:')
print(Prob_Othello)

[-0.889]
Probability of late blight occurence in Othello:
[0.70868376]


**C**

In [351]:
while True:
    # Request user input
    C = input("What is the probability of late blight occurrence in the Columbia Basin of Washington?\n \
                 A) <50% \n \
                 B) >50% \n")
    # If Risk Factor (RF) C is option "A"
    if (C == "A") :
        # Set RF to 0
        C = 0
    # Else, RF is "B" and
    elif (C == "B"):
        # Set it to 30
        C = 4
    # Break
    break

What is the probability of late blight occurrence in the Columbia Basin of Washington?
                  A) <50% 
                  B) >50% 
 B


**D**

In [352]:
while True:
    # Request user input
    D = input("What is the date?\n \
                 A) December through April \n \
                 B) May, July - November \n \
                 C) June \n")
    # If Risk Factor (RF) D is option "A"
    if (D == "A") :
        # Set RF to 0
        D = 0
    # Else, if RF is "B" and
    elif (D == "B"):
        # Set it to 1
        D = 1
    # Else, RF is "C" and
    elif (D == "C"):
        # Set it to 3J
        D = 3J
    # Break
    break

What is the date?
                  A) December through April 
                  B) May, July - November 
                  C) June 
 B


**E**

In [353]:
while True:
    # Request user input
    E = input("How many rainy days are expected to occur within the next 7 days?\n \
                 A) 0 rainy days expected \n \
                 B) 1 rainy days expected \n \
                 C) 2 rainy days expected \n \
                 D) 3 or more rainy days expected\n")
    # If Risk Factor (RF) E is option "A"
    if (E == "A") :
        # Set RF to 1
        E = 1
    # Else, if RF is "B" and
    elif (E == "B"):
        # Set it to 2
        E = 2
    # Else, if RF is "C" and
    elif (E == "C"):
        # Set it to 4
        E = 4
    # Else, if RF is "C" and
    elif (E == "D"):
        # Set it to 5
        E = 5
    # Break
    break

How many rainy days are expected to occur within the next 7 days?
                  A) 0 rainy days expected 
                  B) 1 rainy days expected 
                  C) 2 rainy days expected 
                  D) 3 or more rainy days expected
 B


**Calcualte Risk factor**

In [357]:
Risk = A*B*C*D*E
Risk

0

**Recommendation for the growers**

In [361]:
if Risk == 0:
    print('Recommendation: Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight on regular bases.')
elif Risk >=1 and Risk <3:
    print('Recommendation: Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')
elif Risk >= 4 and Risk <6:
    print('Recommendation:  Apply late blight fungicide before any rainy period. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')
elif Risk >=8 and Risk <11:
    print('Apply late blight fungicide before any rainy periods and continue for 3 wks. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight. Monitor fields for late blight on regular bases.')
elif Risk == 30:
    print('Recommendataion: Confirm occurrence of late blight. If present, apply late blight fungicides on a 5 to 7 day schedule and continue until harvest. Avoid over watering and irrigation during and just after rainy, cool and cloudy weather. Harvest during dry weather. Sort out rotten tubers going into storage. Consult literature on recommendations for management of late blight in the field during late season in infected tubers in storage. ')

Recommendation: Late blight is not likely, fungicide application is not recommended. Dispose of all cull or refuse tubers and manage volunteer potato plants, especially in fields were late blight occurred the last two years. Monitor fields for late blight on regular bases.


### Graveyard

#### Both April and May

In [323]:
os.chdir('C:/Users/sudha.gcupadhaya/Desktop/LateBlight/Data')
df_othello = pd.read_csv('Othello_AM_2020.csv')
df_prosser = pd.read_csv('Prosser_AM_2020.csv')
df_tricities = pd.read_csv('Tricities_AM_2020.csv')

In [324]:
df_othello['Station'] = 'Othello'
df_prosser['Station'] = 'Prosser'
df_tricities['Station'] = 'Tricities'

In [325]:
df_AM = pd.concat([df_othello, df_prosser, df_tricities])
df_AM.head()

,Date,Date.1,Min°C,Avg°C,Max°C,Avg1.5m DP°C,Avg1.5m RH%,Avg1.5m LWu.,AvgDir,Avg Speedm/s,2m MaxGustm/s,5 cm°C,Min°C.1,Avg°C.1,TotPrecmm,TotalSolarRadMJ/m²,ETomm,ETrmm,Station,Avg2m Atm.PresshPa
0,2020/04/01,1,-2.9,3.0,9.5,-2.1,72.0,0.02,W,2.6,7.0,NaN,6.1,7.3,0.0,15.50,1.97,2.64,Othello,NaN
1,2020/04/02,2,-4.3,3.2,10.2,-6.4,53.2,0.01,W,2.4,7.3,NaN,5.3,6.9,0.0,19.53,2.54,3.50,Othello,NaN
2,2020/04/03,3,-2.9,3.8,11.1,-5.3,56.7,0.00,SW,4.5,12.8,NaN,5.7,7.2,0.0,19.06,3.07,4.48,Othello,NaN
3,2020/04/04,4,-3.6,5.0,11.1,-5.4,51.0,0.00,NE,1.7,6.5,NaN,5.6,7.4,0.0,19.59,2.44,3.21,Othello,NaN
4,2020/04/05,5,3.0,7.4,13.5,1.3,68.3,0.00,N,2.4,6.8,NaN,7.6,8.7,0.0,14.91,2.47,3.33,Othello,NaN


In [326]:
df_AM = df_AM[['Date', 'TotPrecmm','Station']]#subset only three columns
df_AM['Month']= pd.DatetimeIndex(df_AM['Date']).month#create a new column for month using date 
df_AM.head()

,Date,TotPrecmm,Station,Month
0,2020/04/01,0.0,Othello,4
1,2020/04/02,0.0,Othello,4
2,2020/04/03,0.0,Othello,4
3,2020/04/04,0.0,Othello,4
4,2020/04/05,0.0,Othello,4


In [327]:
grouped = df_AM.groupby(['Station', 'Month','TotPrecmm']).size().reset_index(name = 'PrecCount')#use groupby function to count no.of rainfall events
grouped.head()

,Station,Month,TotPrecmm,PrecCount
0,Othello,4,0.00,29
1,Othello,4,1.52,1
2,Othello,5,0.00,25
3,Othello,5,0.76,1
4,Othello,5,2.03,1


In [328]:
grouped = grouped.query('TotPrecmm==0')#Subset the rainfall events =0
grouped['TotDays'] = grouped['Month'].apply(lambda x: 30 if x== 4 else 31)#create new column for the total no. of days for both April and may 
grouped['RainyDays'] = grouped['TotDays'] - grouped['PrecCount']#calculate total no. of rainy days
grouped

,Station,Month,TotPrecmm,PrecCount,TotDays,RainyDays
0,Othello,4,0.0,29,30,1
2,Othello,5,0.0,25,31,6
9,Prosser,4,0.0,29,30,1
11,Prosser,5,0.0,22,31,9
20,Tricities,4,0.0,29,30,1
22,Tricities,5,0.0,21,31,10


In [329]:
RainDays_April_Othello = np.array(grouped['RainyDays'][(grouped['Station']=='Othello') & (grouped['Month'] ==4)])#extract no.of rainy days for each month and location
RainDays_May_Othello = np.array(grouped['RainyDays'][(grouped['Station']=='Othello') & (grouped['Month'] ==5)])
RainDays_April_Prosser = np.array(grouped['RainyDays'][(grouped['Station']=='Prosser') & (grouped['Month'] ==4)])
RainDays_May_Prosser = np.array(grouped['RainyDays'][(grouped['Station']=='Prosser') & (grouped['Month'] ==5)])
RainDays_April_Tricities = np.array(grouped['RainyDays'][(grouped['Station']=='Tricities') & (grouped['Month'] ==4)])
RainDays_May_Tricities = np.array(grouped['RainyDays'][(grouped['Station']=='Tricities') & (grouped['Month'] ==5)])